<a href="https://colab.research.google.com/github/ThilakBS/GTech-Seth-Bonder/blob/main/SBCDL_CatsAndDogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Go wild -- this assignment is to recognize Cats and Dogs. The load dataset function is loading the data properly.
# Note that the image are in color, i.e., different from the MNIST and Fashion image. The tensors will be different
# The rest is yours. Here are a few suggestions

# 1. Start with a basic model -- that is your baseline
# 2. Move a convolutional network
# 3. Finish with a convolutional network with data augmentation (see the video)

In [ ]:
# USE ONLY IF using a GPU on PACE

# If this is not imported the GPU on the PACE will not function properly
# otherwise, on another system, it will need built with cuda and loaded
# the same version to work properly
# os.system("module load cuda/12.1.1")
import os
import time
os.system("module load cuda/12.1.1")
time.sleep(20)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow import keras
from keras import layers
from keras.datasets import imdb
import numpy as np
import os, shutil, pathlib
from keras.utils import load_img
from keras.utils import img_to_array
from keras.utils import array_to_img
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
size_picture = 128

In [ ]:
from keras.utils import image_dataset_from_directory

In [ ]:
def tensor_from_image(name):
    img = load_img(name)
    img = img.resize((size_picture,size_picture))
    return (img,img_to_array(img).astype("uint8").reshape((size_picture,size_picture,3)))

In [ ]:
def load_dataset(start_index,end_index):
    images = []
    tensor_inputs = np.zeros((2*(end_index-start_index+1),size_picture,size_picture,3))
    tensor_labels = np.zeros((2*(end_index-start_index+1),1))
    base_directory = pathlib.Path('../datasets/cats_and_dogs/training_set')
    k = 0
    for category in {"cat","dog"}:
        names = [f"{category}.{i}.jpg" for i in range(start_index,end_index+1)]
        for n in names:
            fname = base_directory / category / n
            (img,t) = tensor_from_image(fname)
            images.append(img)
            tensor_inputs[k] = t
            tensor_labels[k] = (category == 'dog')
            k += 1
    return (images,tensor_inputs,tensor_labels)

In [ ]:
def binary_classify(model,inputs,outputs):
  model.compile(optimizer="rmsprop",loss="binary_crossentropy",metrics=["accuracy"])
  model.fit(inputs,outputs,epochs=100)

In [ ]:
(training_images,training_inputs,training_labels) = load_dataset(1,3000)
(testing_images,testing_inputs,testing_labels) = load_dataset(3001,3999)

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/cats_and_dogs/training_set/dog/dog.1.jpg'

In [ ]:
def build_model(training_images):
  data_augmentation = keras.Sequential(
    [
      layers.RandomFlip("horizontal"),
      layers.RandomRotation(0.1),
      layers.RandomZoom(0.2),
    ]
  )
  input = keras.Input(shape=(training_images[1],training_images[2],3),name="data")
  x = data_augmentation(input)
  x = layers.Rescaling(1./255)(x)
  x = layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu')(x)
  x = layers.MaxPooling2D(pool_size = 2)(x)
  x = layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu')(x)
  x = layers.MaxPooling2D(pool_size = 2)(x)
  x = layers.Conv2D(filters = 128, kernel_size = 3, activation = 'relu')(x)
  x = layers.Flatten()(x)
  output = layers.Dense(1, activation = 'sigmoid')(x)
  model = keras.Model(input, output)
  return model

In [ ]:
def train_model(model,inputs,outputs):
  model.compile(optimizer="rmsprop",loss="binary_crossentropy",metrics=["accuracy"])
  model.fit(inputs,outputs,epochs=100)

In [ ]:
model = build_model(training_inputs.shape[1:])


In [ ]:
train_model(model,training_inputs,training_labels)

In [ ]:
model.evaluate(testing_inputs,testing_labels)